# Setup


In [104]:
from setup_general import *
from setup_embedding import *

In [105]:
type_indicators = {}
with open('data/type_indicators/type_ind_cut.txt', 'r') as f:
    for line in f:
        a = line.split('\'')
        type = a[1]
        indicators = a[2].split()
        type_indicators[type] = indicators
save_indicators = {}
with open('data/type_indicators/save_indicator.txt', 'r') as f:
    for line in f:
        a = line.split('\'')
        type = a[1]
        indicators = a[2].split()
        save_indicators[type] = indicators

In [106]:
# naive functions for type from text keywords

def filtering(text):
    pred = []
    for type in types:            
        if type in text:
            pred.append(type)
    if ('drawing' in text) or ('sketch' in text) or ('design' in text):
        pred.append('design/drawing/sketch')
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1
    
def indicating(text):
    pred = []
    for type in types:
        for indicator in type_indicators[type]:
            if indicator in text:
                pred.append(type)
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1

def save_indicating(text):
    pred = []
    for type in types:
        if type in save_indicators.keys():
            for indicator in save_indicators[type]:
                if indicator in text:
                    pred.append(type)
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1


# combine models via class-probability combination (soft-voting)


In [107]:
# is the full ds used for submission?
full = True
# submit to 
sub_name = 'kaspar_type_checks_xgrfnn_no_emb.csv'

In [108]:
#define models to be used for testing use 03 for submission use full
import pickle
xgb = XGBClassifier()
xgb.load_model('models/xg/full_smote100.json')

rf = pickle.load(open('./models/rf/train_prep_full_best' , 'rb'))

boost_emb = XGBClassifier()
boost_emb.load_model('models/nlp/bal_full_xg_emb.json')

nn = TabNetClassifier()
nn.load_model('models/nn/tabnet_full.zip')

/home/till/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [109]:
data = test_prep.copy() if full else train_prep.copy()

features = data.drop('type', axis=1)
labels = data.type

# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)
if full: X_test = features

In [110]:
results = pd.DataFrame()
results['id'] = X_test.index
results.set_index('id', inplace=True)
if not full: results['type'] = y_test

#results['rf'] = rf.predict(X_test)
#results['xg'] = xgb.predict_proba(X_test)
#results['nn'] = nn.predict(X_test.values)

results['filter'] = [-1] * len(results)
results['indi'] = [-1] * len(results)
results['save'] = [-1] * len(results)
results['emb'] = [[-1]] * len(results)


results['xg'] = [[-1]] * len(results)
results['rf'] = [[-1]] * len(results)
results['nn'] = [[-1]] * len(results)

In [111]:
for i,item in enumerate(xgb.predict_proba(X_test)):
    results['xg'].iloc[i] = np.array(item)

for i,item in enumerate(rf.predict_proba(X_test)):
    results['rf'].iloc[i] = np.array(item)

for i,item in enumerate(nn.predict_proba(X_test.values)):
    results['nn'].iloc[i] = np.array(item)


/tmp/ipykernel_9834/1971795648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['xg'].iloc[i] = np.array(item)
/tmp/ipykernel_9834/1971795648.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['rf'].iloc[i] = np.array(item)


KeyError: 'nn'

In [ ]:
text = test_curie.copy() if full else train_curie.copy()

features = text.drop('type', axis=1)
labels = text.type

#text['pred'] = boost_emb.predict_proba(features)
text['pred'] = [[-1]] * len(features)

In [ ]:
for i,item in enumerate(boost_emb.predict_proba(features)):
    text['pred'].iloc[i] = np.array(item)

/tmp/ipykernel_9834/2524059003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['pred'].iloc[i] = np.array(item)


In [ ]:
"""
# establish our own rules to determine type from text - eventually not beneficial
text['filter'] = text.text_features.apply(filtering) # check for the type in the text
text['indicating'] = text.text_features.apply(indicating) # check for other often occuring type indicating words
text['save'] = text.text_features.apply(save_indicating) # only check for words that (almost) only occur with a certain type

text['filter'] = text['filter'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)
text['indicating'] = text['indicating'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)
text['save'] = text['save'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)

for index, item in text.iterrows():
        if index in results.index:
            results.at[index, 'filter'] = item['filter']
            results.at[index, 'indi'] = item['indicating']
            results.at[index, 'save'] = item['save']
            results.at[index, 'emb'] = item['pred']
"""

"\n# establish our own rules to determine type from text - eventually not beneficial\ntext['filter'] = text.text_features.apply(filtering) # check for the type in the text\ntext['indicating'] = text.text_features.apply(indicating) # check for other often occuring type indicating words\ntext['save'] = text.text_features.apply(save_indicating) # only check for words that (almost) only occur with a certain type\n\ntext['filter'] = text['filter'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)\ntext['indicating'] = text['indicating'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)\ntext['save'] = text['save'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)\n\nfor index, item in text.iterrows():\n        if index in results.index:\n            results.at[index, 'filter'] = item['filter']\n            results.at[index, 'indi'] = item['indicating']\n            results.at[index, 'save'] = 

In [ ]:
for index, item in text.iterrows():
        if index in results.index:
            results.at[index, 'emb'] = item['pred']

# evalaluate

In [ ]:
from operator import add
def vote(preds):
    if preds[-1][0] == -1:
        preds = preds[:-1]
    res = np.sum(preds, axis=0)
    return np.argmax(res)

In [ ]:
results['prediction'] = results.apply(lambda row: vote([row.xg, row.rf, row.nn]), axis=1)
if not full:
    print(accuracy_score(results.type, results.prediction))
    print(classification_report(results.type, results.prediction))

TypeError: 'int' object is not subscriptable

In [ ]:
1 == np.array([1,2])

array([ True, False])

# submission

In [ ]:
# convert to numeric
def replace_value(value: str):
    if pd.isnull(value):
        return value
    return np.float64(value.replace(',', '.'))


# convert to numeric and only keep year part
def replace_start_end(value: str):
    if pd.isnull(value):
        return value
    if re.match('^d?ddd$', value):
        return int(value)
    if re.match('dddd$', value):
        return int(value[-4:])
    elif not value[0].isdigit():
        return int(f'19{value[-2:]}')
    else:
        return nan


def extract_year_from_name(row):
    name = row['name']
    start = row['start']
    if pd.isnull(start) and not pd.isnull(name):
        match = re.search('\d\d\d\d', name)
        if match:
            start = match.group()
    return start


def preprocess_dataframe(df, submission=False):
    categorical_cols = ['material', 'location', 'before_Christ', 'country_and_unit', 'technique', 'parameter',
                        'museum_abbr', 'damages', 'state', 'color', 'event_type', 'collection_mark']
    categorical_cols += ['unit', 'participants_role', 'participant', 'musealia_mark']

    # just keeping track what values are used
    numeric_cols = ['start', 'end', 'value', 'collection_queue_nr', 'is_original', 'ks', 'element_count',
                    'musealia_seria_nr', 'musealia_queue_nr']

    dropped_cols = ['id', 'parish']  # can't use
    dropped_cols += ['full_nr', 'class', 'collection_additional_nr', 'additional_text', 'text', 'initial_info',
                     'musealia_additional_nr']  # 'commentary','name', 'legend'

    if not submission: dropped_cols.append('type')

    df['start'] = df['start'].apply(replace_start_end)
    df['end'] = df['end'].apply(replace_start_end)
    df['value'] = df['value'].apply(replace_value)
    df['start'] = df[['name', 'start']].apply(extract_year_from_name, axis=1)

    df = df.drop(columns=dropped_cols)
    df = pd.get_dummies(df, columns=categorical_cols)
    df = df.fillna(0)
    return df

In [ ]:
def extract_label_from_comment(row):
    # comment #################################################
    comment = row['commentary']

    if not pd.isnull(comment):
        comment = str(comment).lower()

        comment_dict = {
            'lakk': 'pitser/templijäljend',
            'must-valge negatiiv': 'fotonegatiiv',
            'pitserilakk': 'pitser/templijäljend',
            'käepide': 'pitsat',
            'перф': 'fotonegatiiv',
            'fotoemulsioon': 'fotomaterjal',
            'plakat':'plakat'
        }
        for key, val in comment_dict.items():
            if comment.startswith(key):
                return val

        if re.match('^\d,\d\d\sg$', comment):
            return 'münt'

        if 'diapositiiv' in comment:
            return 'diapositiiv'

    # name #################################################
    name = row['name']

    if not pd.isnull(name):
        name = str(name).lower()
        if name == ['denaar', 'killing', 'penn', 'schilling', '1/2 örtug', 'dirhem', 'fyrk']:
            return 'münt'

        for val in ['medal', 'plakat', 'märkmed', 'maal', 'kiri', 'kleit', 'kava', 'joonistus', 'graafika', 'dokument',
                    'ajakiri', 'telegramm', 'skulptuur', 'raamat', 'postkaart', 'nukk', 'skulptuur', 'käsikiri']:
            if name.startswith(val):
                return val

        name_dict = {
            'kaustik': 'kaustik/vihik',
            'vihik': 'kaustik/vihik',
            'reprofoto': 'diapositiiv',
        }
        for key, val in name_dict.items():
            if name.startswith(key):
                return val
    return nan

def replace_predictions(labels, pred):
    result = np.array(pred, copy=True)
    for i, label in enumerate(labels):
        if not pd.isnull(label) and label != 0:
            result[i] = label
    return result

In [ ]:
from numpy import nan

if full:
    results.prediction = results.prediction.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
    #results = results.assign(prediction='foto')

    a = results.prediction.copy().tolist()

    df_submission = pd.read_csv("data/general/test.csv")
    x2 = preprocess_dataframe(df_submission, submission=True)
    # reorder columns + add missing columns + remove extra columns
    x2_labels = x2.apply(extract_label_from_comment, axis=1)
    

    results.prediction = replace_predictions(x2_labels, results.prediction)
    submission = pd.DataFrame({'id': results.index ,'type': results.prediction})

    b = results.prediction.copy().tolist()

    count = 0
    for i in range(len(a)):
        if a[i] != b[i]:
            count += 1
            print(a[i], b[i])

    print(count)


    #submission.groupby('type').nunique()  # predicted classes

    submission.to_csv(f'submissions/{sub_name}', index=False)

kutse kiri
käsikiri kiri
foto diapositiiv
dokument kaustik/vihik
foto postkaart
pitsat pitser/templijäljend
kiri postkaart
kiri käsikiri
fotonegatiiv maal
lina/linik kleit
märkmed käsikiri
kiri postkaart
dokument kava
kiri postkaart
kutse kava
kava kiri
postkaart kiri
käsikiri plakat
telegramm kiri
kiri postkaart
aukiri/auaadress raamat
aukiri/auaadress kiri
foto, postkaart postkaart
kiri, postkaart kiri
joonistus graafika
väiketrükis kiri
kiri postkaart
dokument kiri
märkmed käsikiri
kiri dokument
silt/märk pitsat
aukiri/auaadress kiri
foto kiri
dokument telegramm
käsikiri, laul/ vokaalmuusika käsikiri
käsikiri telegramm
käsikiri märkmed
aukiri/auaadress kiri
aukiri/auaadress kiri
kiri postkaart
kaustik/vihik kiri
kutse kiri
graafika joonistus
foto kava
dokument kiri
raamat ajakiri
kiri kava
foto kava
märkmed käsikiri
dokument kiri
telegramm kiri
postkaart kiri
kiri telegramm
käsikiri dokument
postkaart kiri
ajaleht dokument
käsikiri, muusikateos käsikiri
foto postkaart
dokument kiri
